In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import json

import sys

sys.path.append("../")

##################################################################
os.environ["TOKENIZERS_PARALLELISM"] = "false"
# os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3,4,5,6,7"
##################################################################

import logging
from src.utils import logging_utils
from src.utils import env_utils

logger = logging.getLogger(__name__)

logging.basicConfig(
    level=logging.DEBUG,
    format=logging_utils.DEFAULT_FORMAT,
    datefmt=logging_utils.DEFAULT_DATEFMT,
    stream=sys.stdout,
)

import torch
import transformers

logger.info(f"{torch.__version__=}, {torch.version.cuda=}")
logger.info(
    f"{torch.cuda.is_available()=}, {torch.cuda.device_count()=}, {torch.cuda.get_device_name()=}"
)
logger.info(f"{transformers.__version__=}")

/share/u/arnab/miniconda3/envs/connection/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2025-09-10 13:51:15 __main__ INFO     torch.__version__='2.8.0+cu126', torch.version.cuda='12.6'
2025-09-10 13:51:15 __main__ INFO     torch.cuda.is_available()=True, torch.cuda.device_count()=1, torch.cuda.get_device_name()='NVIDIA RTX A6000'
2025-09-10 13:51:15 __main__ INFO     transformers.__version__='4.56.1'


In [3]:
from src.utils.training_utils import get_device_map

# model_key = "meta-llama/Llama-3.2-3B"
# model_key = "meta-llama/Llama-3.1-8B"
# model_key = "meta-llama/Llama-3.1-70B-Instruct"
# model_key = "meta-llama/Llama-3.3-70B-Instruct"
# model_key = "meta-llama/Llama-3.1-405B-Instruct"

# model_key = "google/gemma-2-9b-it"
# model_key = "google/gemma-3-12b-it"
model_key = "google/gemma-2-27b-it"

# model_key = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"

# model_key = "allenai/OLMo-2-1124-7B-Instruct"
# model_key = "allenai/OLMo-7B-0424-hf"

# model_key = "Qwen/Qwen2-7B"
# model_key = "Qwen/Qwen2.5-14B-Instruct"
# model_key = "Qwen/Qwen2.5-32B-Instruct"
# model_key = "Qwen/Qwen2.5-72B-Instruct"

# model_key = "Qwen/Qwen3-1.7B"
# model_key = "Qwen/Qwen3-4B"
# model_key = "Qwen/Qwen3-8B"
# model_key = "Qwen/Qwen3-14B"
# model_key = "Qwen/Qwen3-32B"

# device_map = get_device_map(model_key, 30, n_gpus=8)
# device_map

2025-09-10 13:51:18 git.cmd DEBUG    Popen(['git', 'version'], cwd=/share/u/arnab/Codes/Projects/retrieval/notebooks, stdin=None, shell=False, universal_newlines=False)
2025-09-10 13:51:18 git.cmd DEBUG    Popen(['git', 'version'], cwd=/share/u/arnab/Codes/Projects/retrieval/notebooks, stdin=None, shell=False, universal_newlines=False)
2025-09-10 13:51:18 wandb.docker.auth DEBUG    Trying paths: ['/share/u/arnab/.docker/config.json', '/share/u/arnab/.dockercfg']
2025-09-10 13:51:18 wandb.docker.auth DEBUG    No config file found


In [4]:
from src.models import ModelandTokenizer

# from transformers import BitsAndBytesConfig

mt = ModelandTokenizer(
    model_key=model_key,
    torch_dtype=torch.bfloat16,
    # device_map=device_map,
    device_map="auto",
    # quantization_config = BitsAndBytesConfig(
    #     # load_in_4bit=True
    #     load_in_8bit=True
    # )
    attn_implementation="eager",
)

2025-09-10 13:51:19 src.models WARNING  google/gemma-2-27b-it not found in /share/u/models
If not found in cache, model will be downloaded from HuggingFace to cache directory
2025-09-10 13:51:19 urllib3.connectionpool DEBUG    Starting new HTTPS connection (1): huggingface.co:443
2025-09-10 13:51:19 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /google/gemma-2-27b-it/resolve/main/config.json HTTP/1.1" 200 0


`torch_dtype` is deprecated! Use `dtype` instead!


2025-09-10 13:51:20 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /google/gemma-2-27b-it/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
2025-09-10 13:51:20 urllib3.connectionpool DEBUG    https://huggingface.co:443 "GET /api/models/google/gemma-2-27b-it/tree/main/additional_chat_templates?recursive=False&expand=False HTTP/1.1" 404 64


`torch_dtype` is deprecated! Use `dtype` instead!


2025-09-10 13:51:20 accelerate.utils.modeling INFO     We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


Loading checkpoint shards: 100%|██████████| 12/12 [00:06<00:00,  1.91it/s]

2025-09-10 13:51:27 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /google/gemma-2-27b-it/resolve/main/generation_config.json HTTP/1.1" 200 0
2025-09-10 13:51:27 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /google/gemma-2-27b-it/resolve/main/custom_generate/generate.py HTTP/1.1" 404 0


2025-09-10 13:51:27 accelerate.big_modeling WARNING  Some parameters are on the meta device because they were offloaded to the cpu.
2025-09-10 13:51:27 src.models INFO     loaded model <google/gemma-2-27b-it> | size: 51931.626 MB | dtype: torch.bfloat16 | device: cuda:0


In [197]:
from src.selection.data import SelectOneTask, CountingTask, YesNoTask, SelectFirstTask
from src.selection.data import SelectionSample, CountingSample, YesNoSample

#################################################################################
TASK_CLS = SelectOneTask
# TASK_CLS = CountingTask
# TASK_CLS = YesNoTask
# TASK_CLS = SelectFirstTask
prompt_template_idx = 0
N_DISTRACTORS = 5
OPTION_STYLE = "single_line"
#################################################################################

select_task = TASK_CLS.load(
    path=os.path.join(
        env_utils.DEFAULT_DATA_DIR,
        "selection",
        # "profession.json"
        # "nationality.json"
        "objects.json",
    )
)

print(select_task)

SelectOneTask: (different objects)
Categories: fruit(15), vehicle(15), furniture(15), animal(15), music instrument(15), clothing(15), electronics(15), sport equipment(15), kitchen appliance(15), vegetable(14), building(15), office supply(15), bathroom item(15), flower(15), tree(15), jewelry(15)



In [198]:
from src.functional import predict_next_token, generate_with_patch

sample = select_task.get_random_sample(
    mt=mt, 
    category="fruit",
    filter_by_lm_prediction=True,
    prompt_template_idx=3,
    # n_options=5,
    n_distractors=3,
)
print(f'"{sample.prompt()}" >> {mt.tokenizer.decode(sample.ans_token_id)}')

gen = generate_with_patch(
    mt=mt, 
    inputs=sample.prompt(),
    n_gen_per_prompt=1,
    remove_prefix=True,
    max_new_tokens=20,
    do_sample=False,
)
print(f"Generation:\"{gen[0]}\"")

"Options: Apartment, Mixer, Watermelon, Motorcycle.
Which among these objects mentioned above is a fruit?
Answer:" >>  Watermelon
Generation:" Watermelon.
"


In [199]:
sample.prediction

[PredictedToken(token=' Watermelon', prob=0.984375, logit=23.0, token_id=197201, metadata=None),
 PredictedToken(token=' **', prob=0.015869140625, logit=18.875, token_id=5231, metadata=None),
 PredictedToken(token=' watermelon', prob=0.000545501708984375, logit=15.5, token_id=87171, metadata=None),
 PredictedToken(token='  ', prob=0.0002002716064453125, logit=14.5, token_id=139, metadata=None),
 PredictedToken(token='\n\n', prob=0.0002002716064453125, logit=14.5, token_id=109, metadata=None)]

In [200]:
from src.selection.data import get_counterfactual_samples_interface

counterfactual_sampler = get_counterfactual_samples_interface[select_task.task_name]

print(counterfactual_sampler)

patch_sample, clean_sample = counterfactual_sampler(
    mt=mt,
    task=select_task,
    patch_category="fruit",
    clean_category="vehicle",
    filter_by_lm_prediction=True,
    prompt_template_idx=3,
    option_style=OPTION_STYLE,
    # distinct_options=True,
    # n_options=6
)

# patch_sample.default_option_style = "single_line"
# clean_sample.default_option_style = "numbered"

print("-" * 100)
print(patch_sample.prompt(), ">>", mt.tokenizer.decode(patch_sample.ans_token_id))
print(clean_sample.prompt(), ">>", mt.tokenizer.decode(clean_sample.ans_token_id))

<function get_counterfactual_samples_within_task at 0x7659d8f3afc0>
2025-09-10 15:49:29 src.selection.data INFO     clean_obj_idx=1 | ['Air fryer', 'Tractor', 'Grape', 'Iris', 'Shower', 'Gloves']
type(task)=<class 'src.selection.data.SelectOneTask'>
2025-09-10 15:49:30 src.selection.data INFO     Options: Air fryer, Tractor, Grape, Iris, Shower, Gloves.
Which among these objects mentioned above is a fruit?
Answer:
2025-09-10 15:49:30 src.selection.data INFO     Blueberry | fruit -> Grape | pred=['" Grape"[86502] (p=0.863, logit=21.375)', '" **"[5231] (p=0.133, logit=19.500)', '"  "[139] (p=0.002, logit=15.062)', '"\n\n"[109] (p=0.001, logit=14.812)', '"\n"[108] (p=0.000, logit=13.438)']
2025-09-10 15:49:31 src.selection.data INFO     Options: Air fryer, Tractor, Grape, Iris, Shower, Gloves.
Which among these objects mentioned above is a vehicle?
Answer:
2025-09-10 15:49:31 src.selection.data INFO     Submarine | vehicle -> Tractor | pred=['" Tractor"[131409] (p=0.898, logit=21.250)', '

In [201]:
(
    mt.tokenizer.decode(clean_sample.metadata["track_type_obj_token_id"]),
    mt.tokenizer.decode(clean_sample.ans_token_id),
)

(' Grape', ' Tractor')

In [202]:
patch_sample.prediction, clean_sample.prediction

([PredictedToken(token=' Grape', prob=0.86328125, logit=21.375, token_id=86502, metadata=None),
  PredictedToken(token=' **', prob=0.1328125, logit=19.5, token_id=5231, metadata=None),
  PredictedToken(token='  ', prob=0.00156402587890625, logit=15.0625, token_id=139, metadata=None),
  PredictedToken(token='\n\n', prob=0.001220703125, logit=14.8125, token_id=109, metadata=None),
  PredictedToken(token='\n', prob=0.000308990478515625, logit=13.4375, token_id=108, metadata=None)],
 [PredictedToken(token=' Tractor', prob=0.8984375, logit=21.25, token_id=131409, metadata=None),
  PredictedToken(token=' **', prob=0.0947265625, logit=19.0, token_id=5231, metadata=None),
  PredictedToken(token='\n\n', prob=0.0020904541015625, logit=15.1875, token_id=109, metadata=None),
  PredictedToken(token='  ', prob=0.0016326904296875, logit=14.9375, token_id=139, metadata=None),
  PredictedToken(token='\n', prob=0.0005645751953125, logit=13.875, token_id=108, metadata=None)])

In [203]:
from dataclasses import dataclass
from dataclasses_json import DataClassJsonMixin
from typing import Union

@dataclass
class CounterFactualSamplePair(DataClassJsonMixin):
    patch_sample: Union[SelectionSample, CountingSample, YesNoSample]
    clean_sample: Union[SelectionSample, CountingSample, YesNoSample]

    @staticmethod
    def sample_type_to_class():
        return {
            "selection": SelectionSample,
            "counting": CountingSample,
            "yes_no": YesNoSample,
        }

    def detensorize(self):
        for sample in [self.patch_sample, self.clean_sample]:
            class_name = type(sample).__name__
            type_to_name = {
                "SelectionSample": "selection",
                "CountingSample": "counting",
                "YesNoSample": "yes_no",
            }
            sample.metadata["sample_type"] = type_to_name[class_name]
        self.patch_sample.detensorize()
        self.clean_sample.detensorize()

    @staticmethod
    def from_dict(d):
        sample_type = d["patch_sample"]["metadata"].pop("sample_type")
        sample_cls = CounterFactualSamplePair.sample_type_to_class()[sample_type]
        patch_sample = sample_cls.from_dict(d["patch_sample"])
        sample_type = d["clean_sample"]["metadata"].pop("sample_type")
        sample_cls = CounterFactualSamplePair.sample_type_to_class()[sample_type]
        clean_sample = sample_cls.from_dict(d["clean_sample"])
        return CounterFactualSamplePair(
            patch_sample=patch_sample,
            clean_sample=clean_sample,
        )


cf_pair = CounterFactualSamplePair(
    patch_sample=patch_sample,
    clean_sample=clean_sample,
)
cf_pair.detensorize()
with open("cf_pair_debug.json", "w") as f:
    json.dump(cf_pair.to_dict(), f, indent=2)

In [204]:
with open("cf_pair_debug.json", "r") as f:
    cf_pair_data = json.load(f)

In [205]:
cf_pair_data["patch_sample"]

{'obj': 'Grape',
 'obj_idx': 2,
 'prompt_template': '<_options_>\nWhich among these objects mentioned above is a <_category_>?\nAnswer:',
 'options': ['Air fryer', 'Tractor', 'Grape', 'Iris', 'Shower', 'Gloves'],
 'answer': 'Grape',
 'subj': 'Blueberry',
 'category': 'fruit',
 'prediction': [{'token': ' Grape',
   'prob': 0.86328125,
   'logit': 21.375,
   'token_id': 86502,
   'metadata': None},
  {'token': ' **',
   'prob': 0.1328125,
   'logit': 19.5,
   'token_id': 5231,
   'metadata': None},
  {'token': '  ',
   'prob': 0.00156402587890625,
   'logit': 15.0625,
   'token_id': 139,
   'metadata': None},
  {'token': '\n\n',
   'prob': 0.001220703125,
   'logit': 14.8125,
   'token_id': 109,
   'metadata': None},
  {'token': '\n',
   'prob': 0.000308990478515625,
   'logit': 13.4375,
   'token_id': 108,
   'metadata': None}],
 'ans_token_id': 86502,
 'metadata': {'track_category': 'vehicle',
  'track_type_obj': 'Tractor',
  'track_type_obj_idx': 1,
  'track_type_obj_token_id': 131409

In [206]:
# CountingSample.from_dict(cf_pair_data["patch_sample"])
cf_pair_loaded = CounterFactualSamplePair.from_dict(cf_pair_data)

In [207]:
cf_pair_loaded.patch_sample

SelectionSample(obj='Grape', obj_idx=2, prompt_template='<_options_>\nWhich among these objects mentioned above is a <_category_>?\nAnswer:', options=['Air fryer', 'Tractor', 'Grape', 'Iris', 'Shower', 'Gloves'], answer='Grape', subj='Blueberry', category='fruit', prediction=[PredictedToken(token=' Grape', prob=0.86328125, logit=21.375, token_id=86502, metadata=None), PredictedToken(token=' **', prob=0.1328125, logit=19.5, token_id=5231, metadata=None), PredictedToken(token='  ', prob=0.00156402587890625, logit=15.0625, token_id=139, metadata=None), PredictedToken(token='\n\n', prob=0.001220703125, logit=14.8125, token_id=109, metadata=None), PredictedToken(token='\n', prob=0.000308990478515625, logit=13.4375, token_id=108, metadata=None)], ans_token_id=86502, metadata={'track_category': 'vehicle', 'track_type_obj': 'Tractor', 'track_type_obj_idx': 1, 'track_type_obj_token_id': 131409, 'ques_pos': 26, 'tokenized': {'input_ids': [[2, 6928, 235292, 4252, 110191, 235269, 131409, 235269, 8

In [208]:
print(cf_pair_loaded.patch_sample.prompt(), ">>", mt.tokenizer.decode(cf_pair_loaded.patch_sample.ans_token_id))
cf_pair_loaded.patch_sample.prediction

Options: Air fryer, Tractor, Grape, Iris, Shower, Gloves.
Which among these objects mentioned above is a fruit?
Answer: >>  Grape


[PredictedToken(token=' Grape', prob=0.86328125, logit=21.375, token_id=86502, metadata=None),
 PredictedToken(token=' **', prob=0.1328125, logit=19.5, token_id=5231, metadata=None),
 PredictedToken(token='  ', prob=0.00156402587890625, logit=15.0625, token_id=139, metadata=None),
 PredictedToken(token='\n\n', prob=0.001220703125, logit=14.8125, token_id=109, metadata=None),
 PredictedToken(token='\n', prob=0.000308990478515625, logit=13.4375, token_id=108, metadata=None)]

In [ ]:
sample_save_path = os.path.join(
    env_utils.DEFAULT_RESULTS_DIR,
    "selection",
    "samples",
    "training",
    mt.name.split("/")[-1],
    select_task.task_name,
    "objects"
)

os.makedirs(sample_save_path, exist_ok=True)

In [219]:
from src.functional import free_gpu_cache
from src.selection.data import get_counterfactual_samples_interface
import random

free_gpu_cache()
validation_set = []
validation_limit = 8

counterfactual_sampler = get_counterfactual_samples_interface[select_task.task_name]

while len(validation_set) < validation_limit:
    print(f"sample {len(validation_set)+1} / {validation_limit}")
    patch, clean = counterfactual_sampler(
        mt=mt,
        task=select_task,
        filter_by_lm_prediction=True,
        prompt_template_idx=prompt_template_idx,
        option_style=OPTION_STYLE,
        distinct_options=True,
        n_distractors=N_DISTRACTORS,
        # patch_n_distractors=N_DISTRACTORS,
        # clean_n_distractors=N_DISTRACTORS
        # n_options = random.choice([5])
    )
    validation_set.append((clean, patch))
    cf_pair = CounterFactualSamplePair(
        patch_sample=patch,
        clean_sample=clean,
    )
    cf_pair.detensorize()
    with open(os.path.join(sample_save_path, f"{len(validation_set):05d}.json"), "w") as f:
        json.dump(cf_pair.to_dict(), f, indent=2)

len(validation_set)

sample 1 / 8
2025-09-10 15:55:30 src.selection.data INFO     clean_obj_idx=1 | ['Blender', 'Apartment', 'Hickory', 'Asparagus', 'Yacht', 'Scissors']
type(task)=<class 'src.selection.data.SelectOneTask'>
2025-09-10 15:55:31 src.selection.data INFO     Which object from the following list shares its category with Spinach?
Options: Shampoo, Church, Projector, Saxophone, Pepper, Paperclip.
Answer:
2025-09-10 15:55:31 src.selection.data INFO     Spinach | vegetable -> Pepper | pred=['" Pepper"[43681] (p=0.988, logit=22.625)', '" **"[5231] (p=0.009, logit=17.875)', '"\n\n"[109] (p=0.001, logit=15.062)', '"  "[139] (p=0.000, logit=14.750)', '"\n"[108] (p=0.000, logit=13.938)']
2025-09-10 15:55:32 src.selection.data INFO     Which object from the following list shares its category with Skyscraper?
Options: Blender, Apartment, Hickory, Asparagus, Yacht, Scissors.
Answer:
2025-09-10 15:55:32 src.selection.data INFO     Skyscraper | building -> Apartment | pred=['" Apartment"[57543] (p=0.852, log

8

In [223]:
free_gpu_cache()
validation_set = []
validation_limit = 8

sample_load_path = os.path.join(
    env_utils.DEFAULT_RESULTS_DIR,
    "selection",
    "samples",
    "training",
    mt.name.split("/")[-1],
    select_task.task_name,
    "objects"
)

sample_files = [
    os.path.join(sample_load_path, f)
    for f in os.listdir(sample_load_path)
    if f.endswith(".json")
]
random.shuffle(sample_files)
sample_files = sample_files[:validation_limit]
for sample_file in sample_files:
    with open(sample_file, "r") as f:
        cf_pair_data = json.load(f)
    cf_pair = CounterFactualSamplePair.from_dict(cf_pair_data)
    validation_set.append((cf_pair.clean_sample, cf_pair.patch_sample))

In [224]:
clean, patch = validation_set[5]
print(clean.prompt(), ">>", mt.tokenizer.decode(clean.ans_token_id))
print(patch.prompt(), ">>", mt.tokenizer.decode(patch.ans_token_id))

Which object from the following list shares its category with Pressure cooker?
Options: Scooter, Monkey, Monitor, Sweater, Clarinet, Dishwasher.
Answer: >>  Dishwasher
Which object from the following list shares its category with Dog?
Options: Giraffe, Watermelon, Ottoman, Rose, Kettle, Birch.
Answer: >>  Gira
